In [112]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_auc_score, fbeta_score

In [113]:
#data_preprocessing

def data_preprocessing(dataset:pd.DataFrame):
    '''
    dataset contain this columns:
        'gender':               ['Female' 'Male']
        'SeniorCitizen':        [0 1]
        'Partner':              ['Yes' 'No']
        'Dependents':           ['No' 'Yes']
        'tenure':               int
        'PhoneService':         ['No' 'Yes']
        'MultipleLines':        ['No phone service' 'No' 'Yes']
        'InternetService':      ['DSL' 'Fiber optic' 'No']
        'OnlineSecurity':       ['No' 'Yes' 'No internet service']
        'OnlineBackup':         ['Yes' 'No' 'No internet service']
        'DeviceProtection':     ['No' 'Yes' 'No internet service']
        'TechSupport':          ['No' 'Yes' 'No internet service']
        'StreamingTV':          ['No' 'Yes' 'No internet service']
        'StreamingMovies':      ['No' 'Yes' 'No internet service']
        'Contract':             ['Month-to-month' 'One year' 'Two year']
        'PaperlessBilling':     ['Yes' 'No']
        'PaymentMethod':        ['Electronic check' 'Mailed check' 'Bank transfer (automatic)' 'Credit card (automatic)']
        'MonthlyCharges':       float
        'TotalCharges':         float
        'Churn':                ['No' 'Yes']   // Not necessary
    '''
    df = dataset.copy()

    # Prepare string variables
    df.columns = df.columns.str.lower().str.replace(' ','_')
    string_columns = list(df.dtypes[df.dtypes=='O'].index)
    for col in string_columns:
        df[col] = df[col].str.lower().str.replace(' ','_')

    # drop customerid
    if 'customerid' in df.columns:
        del df['customerid']
        
    # Prepare numeric columns type
    if 'tenure' in df.columns:
        df['tenure'] = pd.to_numeric(df['tenure'], errors='coerce')
        df['tenure'] = df['tenure'].fillna(0)
    else:
        raise Exception("!!! Tenure column didn't exist !!!")

    if 'monthlycharges' in df.columns:
        df['monthlycharges'] = pd.to_numeric(df['monthlycharges'], errors='coerce')
        df['monthlycharges'] = df['monthlycharges'].fillna(0)
    else:
        raise Exception("!!! MonthlyCharges column didn't exist !!!")

    if 'totalcharges' in df.columns:
        df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
        df['totalcharges'] = df['totalcharges'].fillna(0)
    else:
        raise Exception("!!! TotalCharges column didn't exist !!!")
        
    # Make dependent variable numeric
    if 'churn' in df.columns:
        df.churn = (df.churn == 'yes').astype(int)
    
    return df

In [114]:
# FeatureEngineering

def FeatureEngineering(dataset:pd.DataFrame, categorical_variables:list, numerical_variables:list, isTrain:bool, scaleColumns:list):
    """
    dataset: pd.Dataframe
    categorical_variables: list of important categorical columns that we want to use in pur model
    numerical_variables: list of important numerical columns that we want to use in pur model
    isTrain:    if you want to train a model, set it 'True'
                if you want to use for test or deployment set it 'False'
    """
    df = dataset.copy()
    categorical_important = categorical_variables.copy()
    numerical_important = numerical_variables.copy()

    df = df[numerical_important + categorical_important]

    # Dummy variables
    if isTrain:
        def ohe(dataset:pd.DataFrame, categorical_columns:str):
            df = dataset.copy()
            df_info = pd.DataFrame(columns=['col', 'uniques'])
            for col in categorical_columns:
                list_Uniques = df[col].unique()
                df_info.loc[len(df_info)] = [col, ','.join(map(str, list_Uniques))]
                for name in list_Uniques:
                    df[str(col)+'_'+str(name)] = df[col].apply(lambda x : 1 if x==name else 0)
                del df[col]

            df_info.to_csv('assets/ohe_info.csv', index=False)
            return df

        df=ohe(df, categorical_important)
    else:
        df_info = pd.read_csv('assets/ohe_info.csv')
        for i in range(len(df_info)):
            col = df_info.iloc[i,:].col
            list_Uniques = df_info.iloc[i,:].uniques.split(',')
            for name in list_Uniques:
                df[str(col)+'_'+str(name)] =  df[col].apply(lambda x : 1 if x==name else 0)
            del df[col]
    print(df.columns)
    # Feature scaling
    if isTrain:
        scale_info = pd.DataFrame(columns=['col','mean','std'])
        for col in scaleColumns:
            mean_col = df[col].mean()
            std_col = df[col].std()
            df[col] = (df[col]-mean_col)/std_col
            scale_info.loc[len(scale_info)] = [col, mean_col, std_col]
        
        scale_info.to_csv('assets/scale_info.csv', index=False)
        # scaler = StandardScaler()
        # scaler.fit(df)
        # df = scaler.transform(df)
        # df = pd.DataFrame(df, columns=scaler.feature_names_in_)
        # pickle.dump(scaler, open('assets/scaler.pkl','wb'))
    else:
        scale_info = pd.read_csv('assets/scale_info.csv')
        
        for i in range(len(scale_info)):
            col = scale_info.loc[i,'col']
            mean_col = scale_info.loc[i,'mean']
            std_col = scale_info.loc[i,'std']

            df[col] = (df[col]-mean_col)/std_col
        # scaler = pickle.load(open('assets/scaler.pkl','rb'))
        # df = scaler.transform(df)
        # df = pd.DataFrame(df, columns=scaler.feature_names_in_)

    return df

In [115]:
df = pd.read_csv('Churn_prediction.csv')
df.drop_duplicates(inplace=True)

from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(df.loc[:,df.columns!='Churn'],
                                                    df['Churn'],
                                                    test_size=0.2,
                                                    random_state=1)

X_train_full.reset_index(inplace=True,drop=True)

y_train_full = [1 if x == 'Yes' else 0 for x in y_train_full]


In [116]:
# Validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_full,
                                                y_train_full,
                                                test_size=0.2,
                                                random_state=11)

X_train.reset_index(inplace=True,drop=True)

# y_train = [1 if x == 'Yes' else 0 for x in y_train]


check acc Val

In [117]:
# Data preprocessing
# categorical_important = ['contract', 'onlinesecurity', 'techsupport', 'internetservice']
categorical_important = ['gender', 'seniorcitizen', 'partner', 'dependents',
                        'phoneservice', 'multiplelines', 'internetservice',
                        'onlinesecurity', 'onlinebackup', 'deviceprotection',
                        'techsupport', 'streamingtv', 'streamingmovies',
                        'contract', 'paperlessbilling', 'paymentmethod']
numerical_important = ['tenure', 'monthlycharges', 'totalcharges']

X_train = data_preprocessing(X_train)
X_train = FeatureEngineering(X_train,
                            categorical_variables=categorical_important,
                            numerical_variables=numerical_important,
                            isTrain=True,
                            scaleColumns=numerical_important+categorical_important
                            )

# Model
clf_LogisticRegression = LogisticRegression(solver ='liblinear', random_state=1, C=0.5)
clf_LogisticRegression.fit(X_train, y_train)

filename = 'assets/clf_Model.sav'
pickle.dump(clf_LogisticRegression, open(filename, 'wb'))

# acc
X_val = data_preprocessing(X_val)
X_val = FeatureEngineering(X_val,
                            categorical_variables=categorical_important,
                            numerical_variables=numerical_important,
                            isTrain=False,
                            scaleColumns=numerical_important+categorical_important
                            )


filename = 'assets/clf_Model.sav'
clf_LogisticRegression = pickle.load(open(filename, 'rb'))

pred = clf_LogisticRegression.predict(X_val)


roc_auc_score(y_val, pred)

Index(['tenure', 'monthlycharges', 'totalcharges', 'gender_male',
       'gender_female', 'seniorcitizen_0', 'seniorcitizen_1', 'partner_no',
       'partner_yes', 'dependents_no', 'dependents_yes', 'phoneservice_no',
       'phoneservice_yes', 'multiplelines_no_phone_service',
       'multiplelines_no', 'multiplelines_yes', 'internetservice_dsl',
       'internetservice_no', 'internetservice_fiber_optic',
       'onlinesecurity_no', 'onlinesecurity_no_internet_service',
       'onlinesecurity_yes', 'onlinebackup_yes',
       'onlinebackup_no_internet_service', 'onlinebackup_no',
       'deviceprotection_no', 'deviceprotection_yes',
       'deviceprotection_no_internet_service', 'techsupport_yes',
       'techsupport_no', 'techsupport_no_internet_service', 'streamingtv_no',
       'streamingtv_yes', 'streamingtv_no_internet_service',
       'streamingmovies_yes', 'streamingmovies_no_internet_service',
       'streamingmovies_no', 'contract_month-to-month', 'contract_two_year',
       '

KeyError: 'gender'

In [ ]:
# Data preprocessing
# categorical_important = ['contract', 'onlinesecurity', 'techsupport', 'internetservice']
categorical_important = ['gender', 'seniorcitizen', 'partner', 'dependents',
                        'phoneservice', 'multiplelines', 'internetservice',
                        'onlinesecurity', 'onlinebackup', 'deviceprotection',
                        'techsupport', 'streamingtv', 'streamingmovies',
                        'contract', 'paperlessbilling', 'paymentmethod']
numerical_important = ['tenure', 'monthlycharges', 'totalcharges']

X_train_full = data_preprocessing(X_train_full)
X_train_full = FeatureEngineering(X_train_full,
                            categorical_variables=categorical_important,
                            numerical_variables=numerical_important,
                            isTrain=True,
                            scaleColumns=numerical_important+categorical_important
                            )

# Model
clf_LogisticRegression = LogisticRegression(solver ='liblinear', random_state=1, C=0.5)
clf_LogisticRegression.fit(X_train_full, y_train_full)

filename = 'assets/clf_Model.sav'
pickle.dump(clf_LogisticRegression, open(filename, 'wb'))

# acc
X_test = data_preprocessing(X_test)
X_test = FeatureEngineering(X_test,
                            categorical_variables=categorical_important,
                            numerical_variables=numerical_important,
                            isTrain=False,
                            scaleColumns=numerical_important)


filename = 'assets/clf_Model.sav'
clf_LogisticRegression = pickle.load(open(filename, 'rb'))

pred = clf_LogisticRegression.predict(X_test)


roc_auc_score(y_test, pred)

0.7487487406155546